In [ ]:
import scipy.io as spio
from scipy.sparse.csgraph import shortest_path
import matplotlib.pyplot as plt
import matplotlib.image as Image
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import random as rand 
import numpy as np
import networkx as nx
import pandas as pd
import math
from PIL import Image



## Order of faces using ISOMAP

In [ ]:
## load data
image_load = spio.loadmat('data/isomap.mat', squeeze_me=True)
image = image_load['images'].T
m, n = image.shape


Step 1: build a weighted graph A using nearest neighbors

In [ ]:
## Build Adjacent matrix with multiple tuning parameters
epsilon = [5,10,15,20,25,30,35,40]
#epsilon = [10, 12, 14, 16, 18, 20, 22, 24, 28]

A = np.zeros(shape = (m,m))
matrixsum = pd.DataFrame(columns = ("epsilon", "matrix sum"))
for e in epsilon:
    for i in range(m):
        for j in range(m):
            distance = np.linalg.norm(image[i]-image[j], ord=2) # $\|x^i-x^j\|$
            A[i,j] = distance
            if distance <=e:
                A[i,j] = distance
            else:
                A[i,j] = 0
    D = shortest_path(A)
    matrixsum.loc[len(matrixsum.index)] = e, D.sum()

In [ ]:
matrix_sum = matrixsum.set_index("epsilon")
matrix_sum.plot()
plt.savefig('images/matrixelbow_large.png')

In [ ]:
## adjacency matrix with e=16

A = np.zeros(shape = (m,m))
matrixsum = pd.DataFrame(columns = ("epsilon", "matrix sum"))
for e in epsilon:
    for i in range(m):
        for j in range(m):
            distance = np.linalg.norm(image[i]-image[j]) # $\|x^i-x^j\|$
            A[i,j] = distance
            if distance <=16:
                A[i,j] = distance
            else:
                A[i,j] = 0


ISOMAP algorithm

In [ ]:
epsilon = [12, 14, 16, 18, 20, 22]

for e in epsilon:
## import image
    image_load = spio.loadmat('data/isomap.mat', squeeze_me=True)
    image = image_load['images'].T
    m, n = image.shape
    
## Step 1: Build adjacency matrix
    A = np.zeros(shape = (m,m))
    for i in range(m):
        for j in range(m):
            distance = np.linalg.norm(image[i]-image[j]) # $\|x^i-x^j\|$
            A[i,j] = distance
            if distance <=e:
                A[i,j] = distance
            else:
                A[i,j] = 0

## Step 2: compute pairwise shortest distance matrix D
    D = shortest_path(A)
    
## Step 3: Use Centering Matrix H to get C
    I = np.identity(m)
    one_matrix = np.ones(shape = 698)
    H = I - (1/m) * np.outer(one_matrix, one_matrix.T)
    C = (-1/2) * np.linalg.multi_dot([H, D**2, H])

## Step 4: Compute leading eigenvectors and eigenvalues
    # compute eigenvalues
    U, sigma, VT = np.linalg.svd(C)

    # Extract first 2 eigenvectors
    dim_1 = U[:,0]*np.sqrt(sigma[0])
    dim_2 = U[:,1]*np.sqrt(sigma[1])
    
## Graph results
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.scatter(dim_1, dim_2)
    
## with images
## https://fcpython.com/visualisation/creating-scatter-plots-with-club-logos-in-python
## https://www.programcreek.com/python/example/102442/matplotlib.offsetbox.OffsetImage

    # select random
    sample = rand.sample(range(m), int(m/15))

    for i_img in sample:
        img_disp = np.reshape(image[i_img,:], (64,64)).T
        
        ab = AnnotationBbox(OffsetImage(img_disp, cmap=plt.cm.gray_r, zoom=0.5), (dim_1[i_img], dim_2[i_img]), pad=0, frameon=True)
        ax.add_artist(ab)
        ax.scatter(dim_1, dim_2)
    
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html
    fig.savefig('images/face_epsilon_'+str(e)+'.png')

In [ ]:
PCA Analysis

In [ ]:
image_mean = np.mean(image, axis=0)

X = image.T - image_mean[:,None] ## Calculate Mean Center

# Find SVD
U, Sigma, VT = np.linalg.svd(X)

# Find dimensions
dim_1 = np.dot(U[:,0].T,X)/np.sqrt(Sigma[0])
dim_2 = np.dot(U[:,1].T,X)/np.sqrt(Sigma[1])

k_approx = 2 # number of components

# Plot the components of the first k_approx=2 singular vectors
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(dim_1, dim_2)


sample = rand.sample(range(m), int(m/15))

for i_img in sample:
    img_disp = np.reshape(image[i_img,:], (64,64)).T

    ab = AnnotationBbox(OffsetImage(img_disp, cmap=plt.cm.gray_r, zoom=0.5), (dim_1[i_img], dim_2[i_img]), pad=0, frameon=True)
    ax.add_artist(ab)
    ax.scatter(dim_1, dim_2)
    
fig.savefig('images/face_PCA.png')

## Eigenfaces and simple face recognition

(a) Perform analysis on the Yale face dataset

In [2]:
pic_path_01 = ['data/yalefaces/subject01.glasses.gif',
              'data/yalefaces/subject01.happy.gif',
              'data/yalefaces/subject01.leftlight.gif',
              'data/yalefaces/subject01.noglasses.gif',
              'data/yalefaces/subject01.normal.gif',
              'data/yalefaces/subject01.rightlight.gif',
              'data/yalefaces/subject01.sad.gif',
              'data/yalefaces/subject01.sleepy.gif',
              'data/yalefaces/subject01.surprised.gif',
               'data/yalefaces/subject01.wink.gif']
pic_path_02 = ['data/yalefaces/subject02.glasses.gif',
              'data/yalefaces/subject02.happy.gif',
              'data/yalefaces/subject02.leftlight.gif',
              'data/yalefaces/subject02.noglasses.gif',
              'data/yalefaces/subject02.normal.gif',
              'data/yalefaces/subject02.rightlight.gif',
              'data/yalefaces/subject02.sad.gif',
              'data/yalefaces/subject02.sleepy.gif',
              'data/yalefaces/subject02.wink.gif']

def read_img(path):

    img = Image.open(path)
    new_image = img.resize((500, 500))

#### (a)

In [3]:
pictures = [pic_path_02]

def pca_image(picture_list, S):

    S01 = np.empty(shape = [10, 4800])
## read in images to np array
    for pic in picture_list:
        for i, i_pic in enumerate(pic):
            image = Image.open(i_pic)
            a, b = image.size
            image2 = image.resize((int(a/4), int(b/4)))
            c, d = image2.size
            vector_image = np.array(image2, dtype = 'int32').flatten()
            S01[i] = vector_image

## Calculate Mean Center
    S01_mean = np.mean(S01, axis=0)
    X_S01 = S01 - S01_mean 

    # Find SVD
    U, sigma, VT = np.linalg.svd(X_S01.T)

    k = 6 # number of components
    # plot each face
    for i in range(k):
        image_final = U[i,:].reshape((int(b/4),int(a/4)))
       # plt.imsave((S +str(i)+'.png'),image_final, cmap = 'gray')
    return (U[:,0:6], S01_mean)



#### (b)

In [76]:
## top eigenvalues and means for each subject
s1_eigen, s1_mean = pca_image([pic_path_01], "images/face01_")
s2_eigen, s2_mean = pca_image([pic_path_02], "images/face02_")

In [5]:
test_paths = ['data/yalefaces/subject01-test.gif','data/yalefaces/subject02-test.gif']    

test_pics = []
for path in test_paths:
    image = Image.open(path)
    a, b = image.size
    image2 = image.resize((int(a/4), int(b/4)))
    c, d = image2.size
    vector_image = np.array(image2, dtype = 'int32').flatten()
    test_pics.append(vector_image)

s1_test_a = test_pics[0]
s2_test_a = test_pics[1]

In [101]:
s1_test_mean = np.mean(s1_test_a, axis=0)
s1_test = s1_test_a - s1_test_mean

s2_test_mean = np.mean(s2_test_a, axis=0)
s2_test = s2_test_a - s2_test_mean

In [125]:
def face_rec(subject, eigen, pc):
    norm=0
    for i in range(pc):
        a = eigen[:,i]
        b = np.linalg.norm(subject - a*(np.dot(a, subject)))**2
        norm+=b
    return norm/pc

In [131]:
import pandas as pd

df = pd.DataFrame(columns = ('PC', "s11", "s21", "Subject01 Correct", "s22", "s12", "Subject02 Correct"))

for i in range(1,7):
    s11 = round(face_rec(s1_test, s1_eigen, i))
    s21 = round(face_rec(s2_test, s1_eigen, i))

    s12 = round(face_rec(s1_test, s2_eigen, i))
    s22 = round(face_rec(s2_test, s2_eigen, i))
    
    if s11 <= s21:
        s1 = "Yes"
    else:
        s1 = "No"
    if s22 <= s12:   
        s2 = "Yes"
    else:
        s2 = "No"
    
    df.loc[len(df.index)] = i, s11, s21, s1, s22, s12, s2

print(df.to_latex())

\begin{tabular}{lrrrlrrl}
\toprule
 & PC & s11 & s21 & Subject01 Correct & s22 & s12 & Subject02 Correct \\
\midrule
0 & 1 & 30805641 & 34716020 & Yes & 24357243 & 23102451 & No \\
1 & 2 & 32006634 & 32938313 & Yes & 29800564 & 28193454 & No \\
2 & 3 & 32315234 & 33637741 & Yes & 29317829 & 29507685 & Yes \\
3 & 4 & 32615887 & 33993706 & Yes & 30440191 & 30506183 & Yes \\
4 & 5 & 32566797 & 34273402 & Yes & 31429445 & 31107151 & No \\
5 & 6 & 32666958 & 34473786 & Yes & 32023531 & 31472330 & No \\
\bottomrule
\end{tabular}

